In [1]:
import pandas as pd
from urllib.request import Request, urlopen
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from datetime import datetime

In [2]:
nltk.download('vader_lexicon')
num_headlines = 3
tickers = ['AAPL', 'TSLA', 'AMZN']
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [3]:
def fetch_news(ticker):
    finviz_url = 'https://finviz.com/quote.ashx?t='
    url = finviz_url + ticker
    req = Request(url=url, headers={'user-agent': 'Mozilla/5.0'})
    response = urlopen(req)
    df = pd.read_html(response.read(), attrs={'id': 'news-table'})[0]
    df.columns = ['Datetime', 'Headline']
    return df.head(num_headlines)

In [4]:
def sentiment_analysis(df):
    df['Sentiment'] = df['Headline'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
    return df

In [5]:
for ticker in tickers:
    news_df = fetch_news(ticker)
    news_df['Ticker'] = ticker
    sentiment_df = sentiment_analysis(news_df)
    print('\nRecent News Headlines and Sentiment for {}: '.format(ticker))
    print(sentiment_df)


Recent News Headlines and Sentiment for AAPL: 
            Datetime                                           Headline  \
0  Feb-05-24 08:39PM  With new iPads around the corner, how will App...   
1            08:29PM  Chinese suppliers make up majority of Vision P...   
2            06:01PM  20 Best Permanent Residency Countries in the W...   

  Ticker  Sentiment  
0   AAPL     0.0000  
1   AAPL     0.2500  
2   AAPL     0.6369  

Recent News Headlines and Sentiment for TSLA: 
            Datetime                                           Headline  \
0  Feb-05-24 11:03PM  Chewing Over the Latest Updates From Microsoft...   
1            09:46PM  Tesla's board needs to keep Elon Musk's buy in...   
2            09:45PM  Tesla's board needs to get rid of 3-4 members,...   

  Ticker  Sentiment  
0   TSLA        0.0  
1   TSLA        0.0  
2   TSLA        0.0  

Recent News Headlines and Sentiment for AMZN: 
            Datetime                                           Headline  \
0  

In [6]:
mean_sentiments = {ticker: sentiment_analysis(fetch_news(ticker))['Sentiment'].mean() for ticker in tickers}
sentiments_df = pd.DataFrame(list(mean_sentiments.items()), columns=['Ticker', 'Mean Sentiment'])
sentiments_df = sentiments_df.set_index('Ticker').sort_values('Mean Sentiment', ascending=False)
print('\nMean Sentiment Scores for Each Ticker:')
print(sentiments_df)


Mean Sentiment Scores for Each Ticker:
        Mean Sentiment
Ticker                
AAPL          0.295633
AMZN          0.186933
TSLA          0.000000
